In [2]:
import pandas as pd  # requires: pip install pandas
import torch
from chronos import BaseChronosPipeline

# Load the AI model
pipeline = BaseChronosPipeline.from_pretrained(
    "amazon/chronos-bolt-tiny",
    device_map="cpu",  # use "cpu" for CPU inference and "mps" for Apple Silicon
    torch_dtype=torch.bfloat16,
)


In [8]:
# make predictions
import pandas as pd
import numpy as np
import torch

# Generate 1 day of minute-by-minute temperature data
timestamps = pd.date_range(start="2023-01-01", periods=1440, freq="T")  # 1440 minutes = 1 day
base_temp = 90  # Starting temperature
trend = np.linspace(0, 10, len(timestamps))  # 10°C increase over the day
noise = np.random.normal(0, 0.2, size=len(timestamps))  # Small fluctuations
temperature = base_temp + trend + noise

# Wrap in a minimal DataFrame for a single item
df = pd.DataFrame({
    "item_id": "boiler_1",   # only one time series
    "timestamp": timestamps,
    "target": temperature
})

# context must be either a 1D tensor, a list of 1D tensors,
# or a left-padded 2D tensor with batch as the first dimension
# Chronos-Bolt models generate quantile forecasts, so forecast has shape
# [num_series, num_quantiles, prediction_length].
forecast = pipeline.predict(
    context=torch.tensor(df["target"]), prediction_length=20
)
forecast[0][0].tolist()

C:\Users\G4G1\AppData\Local\Temp\ipykernel_7652\2262547877.py:7: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  timestamps = pd.date_range(start="2023-01-01", periods=1440, freq="T")  # 1440 minutes = 1 day


[99.69197845458984,
 99.66931915283203,
 99.64665222167969,
 99.64665222167969,
 99.69197845458984,
 99.69197845458984,
 99.66931915283203,
 99.69197845458984,
 99.71463775634766,
 99.73729705810547,
 99.75996398925781,
 99.78262329101562,
 99.82794189453125,
 99.80528259277344,
 99.78262329101562,
 99.75996398925781,
 99.78262329101562,
 99.73729705810547,
 99.75996398925781,
 99.75996398925781]